# GSPhar Model Notebook
This notebook recreates the GSPhar model from the original `d-GSPHAR.ipynb`, including dataset utilities, model definition, training functions, and a summary display.

In [2]:
!pip install torchinfo

In [3]:
# Imports
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from scipy.linalg import sqrtm, eig
from torchinfo import summary

In [4]:
# Spillover index utility (from original notebook)
def compute_spillover_index(data, horizon, lag, scarcity_prop, standardized=True):
    data_array = data.values
    from statsmodels.tsa.api import VAR
    model = VAR(data_array)
    results = model.fit(maxlags=lag)
    Sigma = results.sigma_u
    A = results.orth_ma_rep(maxn=horizon-1)
    Sigma_A = []
    A_Sigma_A = []
    for h in range(horizon):
        SA = (A[h] @ Sigma @ np.linalg.inv(np.diag(np.sqrt(np.diag(Sigma)))))**2
        Sigma_A.append(SA)
        ASA = A[h] @ Sigma @ A[h].T
        A_Sigma_A.append(ASA)
    num = np.cumsum(Sigma_A, axis=0)
    den = np.cumsum(A_Sigma_A, axis=0)
    gfevd = np.array([num[h]/np.diag(den[h])[:,None] for h in range(horizon)])
    if standardized:
        gfevd = np.array([m/m.sum(axis=1,keepdims=True) for m in gfevd])
    spill = gfevd[-1].T*100
    df = pd.DataFrame(spill, index=results.names, columns=results.names)
    thresh = pd.Series(df.values.flatten()).quantile(scarcity_prop)
    df[df<thresh] = 0
    mat = df.values; np.fill_diagonal(mat,0)
    return mat/df.shape[0] if standardized else mat

In [5]:
# GSPhar model definition
class GSPHAR(nn.Module):
    def __init__(self, input_dim, output_dim, filter_size, A):
        super(GSPHAR, self).__init__()
        self.A = torch.from_numpy(A).float()
        self.filter_size = filter_size
        self.conv1d_lag5 = nn.Conv1d(filter_size, filter_size, 5, groups=filter_size, bias=False)
        nn.init.constant_(self.conv1d_lag5.weight, 1.0/5)
        self.conv1d_lag22 = nn.Conv1d(filter_size, filter_size, 22, groups=filter_size, bias=False)
        nn.init.constant_(self.conv1d_lag22.weight, 1.0/22)
        self.spatial_process = nn.Sequential(
            nn.Linear(2,16), nn.ReLU(), nn.Linear(16,16), nn.ReLU(), nn.Linear(16,1), nn.ReLU()
        )
        self.linear_output = nn.Linear(input_dim, output_dim)

    def nomalized_magnet_laplacian(self, A, q, norm=True):
        A_s = (A + A.T)/2
        D = np.diag(A_s.sum(axis=1))
        theta = 2*np.pi*q*(A-A.T)
        if norm:
            D_inv_sqrt = sqrtm(np.linalg.inv(D))
            L = np.eye(A_s.shape[0]) - (D_inv_sqrt @ A_s @ D_inv_sqrt)*np.exp(1j*theta)
        else:
            L = D - A_s*np.exp(1j*theta)
        return L

    def dynamic_magnet_Laplacian(self, A, xp, xq):
        A_diff = (A - A.T).clamp(min=0)
        def corr(x):
            xm = x - x.mean(dim=2,keepdim=True)
            cov = torch.bmm(xm, xm.transpose(1,2))/(x.shape[2]-1)
            sd = xm.std(dim=2,unbiased=True).clamp(min=1)
            return cov/(sd.unsqueeze(2)*sd.unsqueeze(1))
        cp, cq = corr(xp), corr(xq)
        W = 0.5*(cp.abs()+cq.abs())*A_diff
        W = F.softmax(W, dim=1)
        Udg, U = [], []
        for i in range(W.size(0)):
            L = self.nomalized_magnet_laplacian(W[i].cpu().numpy(),0.25)
            vals, vecs = eig(L)
            idx = np.argsort(vals.real)
            Udg.append(torch.tensor(vecs.real[:,idx],dtype=torch.cfloat))
            U.append(torch.tensor(vecs.real.T[idx],dtype=torch.cfloat))
        return torch.stack(Udg), torch.stack(U)

    def forward(self, x1, x5, x22):
        device = x1.device
        A = self.A.to(device)
        Udg, U = self.dynamic_magnet_Laplacian(A, x5, x22)
        Udg, U = Udg.to(device), U.to(device)
        # spectral transform and conv on lag5
        x5c = torch.complex(x5,torch.zeros_like(x5))
        x5s = torch.matmul(Udg, x5c)
        w5 = F.softmax(torch.exp(self.conv1d_lag5.weight),dim=-1)
        r5 = F.conv1d(x5s.real, w5, groups=self.filter_size)
        i5 = F.conv1d(x5s.imag, w5, groups=self.filter_size)
        x5f = torch.complex(r5,i5).squeeze(-1)
        # similarly for lag22
        x22c = torch.complex(x22,torch.zeros_like(x22))
        x22s = torch.matmul(Udg, x22c)
        w22 = F.softmax(torch.exp(self.conv1d_lag22.weight),dim=-1)
        r22 = F.conv1d(x22s.real, w22, groups=self.filter_size)
        i22 = F.conv1d(x22s.imag, w22, groups=self.filter_size)
        x22f = torch.complex(r22,i22).squeeze(-1)
        # lag1
        x1c = torch.complex(x1,torch.zeros_like(x1)).unsqueeze(-1)
        x1f = torch.matmul(Udg, x1c).squeeze(-1)
        spec = torch.stack((x1f, x5f, x22f),dim=-1)
        yr = self.linear_output(spec.real)
        yi = self.linear_output(spec.imag)
        ysp = torch.complex(yr, yi)
        yout = torch.matmul(U, ysp)
        ystack = torch.stack((yout.real.squeeze(-1),yout.imag.squeeze(-1)),dim=-1)
        return self.spatial_process(ystack).squeeze(-1)

In [6]:
# Instantiate and display summary
filter_size=24; input_dim=3; output_dim=1
A_eye = np.eye(filter_size)
model = GSPHAR(input_dim,output_dim,filter_size,A_eye)
summary(model, input_data=(torch.randn(1,filter_size),torch.randn(1,filter_size,5),torch.randn(1,filter_size,22)), col_names=["input_size","output_size","num_params"], depth=3)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #
GSPHAR                                   [1, 24]                   [1, 24]                   648
├─Linear: 1-1                            [1, 24, 3]                [1, 24, 1]                4
├─Linear: 1-2                            [1, 24, 3]                [1, 24, 1]                (recursive)
├─Sequential: 1-3                        [1, 24, 2]                [1, 24, 1]                --
│    └─Linear: 2-1                       [1, 24, 2]                [1, 24, 16]               48
│    └─ReLU: 2-2                         [1, 24, 16]               [1, 24, 16]               --
│    └─Linear: 2-3                       [1, 24, 16]               [1, 24, 16]               272
│    └─ReLU: 2-4                         [1, 24, 16]               [1, 24, 16]               --
│    └─Linear: 2-5                       [1, 24, 16]               [1, 24, 1]                17
│    └─ReLU: 2-6         